In [1]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package punkt to /Users/rohini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rohini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rohini/nltk_data...


True

In [2]:
data=pd.read_csv(r'job_description.csv')

In [3]:
data=data.drop(columns=['Unnamed: 0','Benefits','Requirement','Requirements'],axis=1)

In [4]:
data

,Category,Description
0,Business Analyst,Description\nJob Title: Junior Business Analys...
1,Business Analyst,Description\nThe Business Analyst (BA) will ca...
2,Business Analyst,Description\nWe help our clients design and de...
3,Business Analyst,Description\nFounded in 2015 and rapidly expan...
4,Business Analyst,Description\nWe are seeking a skilled Business...
...,...,...
320,UI/UX,Description\nThe UX Designer will be focused o...
321,UI/UX,Description\nWe are seeking a dynamic UI/UX de...
322,UI/UX,Description\nWe are looking for a UI/UX Design...
323,UI/UX,Description\nRemote – Full-Time\nClay is a glo...


In [5]:
data['Category'].unique()

array(['Business Analyst', 'Cloud', 'Data Scientist', 'HR',
       'Software Developer', 'UI/UX'], dtype=object)

In [6]:
# Get unique values in the "Category" column
unique_categories = data['Category'].unique()

# Create a dictionary to store datasets for each category
category_datasets = {}

# Iterate over unique categories
for category in unique_categories:
    # Select rows with the current category
    category_df = data[data['Category'] == category]
    # Create a new DataFrame containing both "Category" and "Description" columns
    category_description_df = category_df[['Category', 'Description']]
    # Store the DataFrame in the dictionary
    category_datasets[category] = category_description_df

# Print the first few rows of each dataset
for category, dataset in category_datasets.items():
    print(f"{category}:")
    print(dataset)  # Print the first three rows of each dataset
    print("----")

Business Analyst:
            Category                                        Description
0   Business Analyst  Description\nJob Title: Junior Business Analys...
1   Business Analyst  Description\nThe Business Analyst (BA) will ca...
2   Business Analyst  Description\nWe help our clients design and de...
3   Business Analyst  Description\nFounded in 2015 and rapidly expan...
4   Business Analyst  Description\nWe are seeking a skilled Business...
5   Business Analyst  Description\nWe are delighted to announce that...
6   Business Analyst  Description\nWe are seeking a consultative Jun...
7   Business Analyst  Description\nIf you are a detail-oriented, cre...
8   Business Analyst  Description\nTitle: Business Analyst\nLocation...
9   Business Analyst  Description\nJob Title: Junior Business Analys...
10  Business Analyst  Description\nThe Business Analyst (BA) will ca...
11  Business Analyst  Description\nWe help our clients design and de...
12  Business Analyst  Description\nFounded in 

In [7]:
data_BA=category_datasets['Business Analyst']
data_cloud=category_datasets['Cloud']
data_DS=category_datasets['Data Scientist']
data_hr=category_datasets['HR']
data_SD=category_datasets['Software Developer']
data_uiux=category_datasets['UI/UX']

In [8]:
# Drop rows with NaN values in the 'Description' column
data_BA.isnull().sum()

Category       0
Description    1
dtype: int64

In [9]:
data_BA=data_BA.dropna(subset=['Description'])

In [10]:
data_BA

,Category,Description
0,Business Analyst,Description\nJob Title: Junior Business Analys...
1,Business Analyst,Description\nThe Business Analyst (BA) will ca...
2,Business Analyst,Description\nWe help our clients design and de...
3,Business Analyst,Description\nFounded in 2015 and rapidly expan...
4,Business Analyst,Description\nWe are seeking a skilled Business...
5,Business Analyst,Description\nWe are delighted to announce that...
6,Business Analyst,Description\nWe are seeking a consultative Jun...
7,Business Analyst,"Description\nIf you are a detail-oriented, cre..."
8,Business Analyst,Description\nTitle: Business Analyst\nLocation...
9,Business Analyst,Description\nJob Title: Junior Business Analys...


In [11]:
# Preprocess the text data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Apply preprocessing to the Description column
data_BA['Description'] = data_BA['Description'].apply(preprocess_text)

/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/1811990132.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_BA['Description'] = data_BA['Description'].apply(preprocess_text)


In [12]:
data_BA

,Category,Description
0,Business Analyst,"[description, job, title, junior, business, an..."
1,Business Analyst,"[description, business, analyst, ba, capture, ..."
2,Business Analyst,"[description, help, clients, design, deliver, ..."
3,Business Analyst,"[description, founded, rapidly, expanding, fre..."
4,Business Analyst,"[description, seeking, skilled, business, anal..."
5,Business Analyst,"[description, delighted, announce, application..."
6,Business Analyst,"[description, seeking, consultative, junior, b..."
7,Business Analyst,"[description, detailoriented, creative, proble..."
8,Business Analyst,"[description, title, business, analyst, locati..."
9,Business Analyst,"[description, job, title, junior, business, an..."


In [13]:
# Create bigrams and trigrams
bigram = Phrases(data_BA['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_BA['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_BA['Description'] = [bigram_mod[doc] for doc in data_BA['Description']]
data_BA['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_BA['Description']]

# Create a dictionary representation of the documents
dictionary_BA= corpora.Dictionary(data_BA['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_BA= [dictionary_BA.doc2bow(doc) for doc in data_BA['Description']]

# Train the LDA model
lda_model_BA= LdaModel(bow_corpus_BA, num_topics=3, id2word=dictionary_BA, passes=10)

# Print the topics
for idx, topic in lda_model_BA.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')

/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/681428599.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_BA['Description'] = [bigram_mod[doc] for doc in data_BA['Description']]
/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/681428599.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_BA['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_BA['Description']]


Topic: 0 
Words: 0.019*"business" + 0.014*"work" + 0.010*"process" + 0.009*"ryanair" + 0.008*"team" + 0.007*"support" + 0.007*"new" + 0.005*"data" + 0.005*"role" + 0.005*"teams"

Topic: 1 
Words: 0.022*"business" + 0.012*"team" + 0.011*"project" + 0.010*"clients" + 0.009*"development" + 0.008*"analyst" + 0.008*"management" + 0.008*"projects" + 0.007*"client" + 0.007*"teams"

Topic: 2 
Words: 0.027*"business" + 0.023*"data" + 0.011*"requirements" + 0.008*"analyst" + 0.008*"technical" + 0.007*"team" + 0.006*"healthcare" + 0.006*"clients" + 0.006*"teams" + 0.006*"systems"



In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_BA= gensimvis.prepare(lda_model_BA, bow_corpus_BA, dictionary_BA)

# Display the visualization
pyLDAvis.display(lda_vis_data_BA)


/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/rohini/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas re

**FOR CLOUD**

In [15]:
# Preprocess the text data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Apply preprocessing to the Description column
data_cloud['Description'] = data_cloud['Description'].apply(preprocess_text)

In [16]:
# Create bigrams and trigrams
bigram = Phrases(data_cloud['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_cloud['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_cloud['Description'] = [bigram_mod[doc] for doc in data_cloud['Description']]
data_cloud['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_cloud['Description']]

# Create a dictionary representation of the documents
dictionary_cloud= corpora.Dictionary(data_cloud['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_cloud= [dictionary_cloud.doc2bow(doc) for doc in data_cloud['Description']]

# Train the LDA model
lda_model_cloud= LdaModel(bow_corpus_cloud, num_topics=3, id2word=dictionary_cloud, passes=10)

# Print the topics
for idx, topic in lda_model_cloud.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')
    
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_cloud= gensimvis.prepare(lda_model_cloud, bow_corpus_cloud, dictionary_cloud)

# Display the visualization
pyLDAvis.display(lda_vis_data_cloud)


Topic: 0 
Words: 0.012*"software" + 0.011*"cloud" + 0.011*"work" + 0.010*"solutions" + 0.007*"services" + 0.007*"team" + 0.006*"architecture" + 0.006*"technology" + 0.005*"design" + 0.005*"description"

Topic: 1 
Words: 0.022*"data" + 0.013*"cloudline" + 0.009*"operations" + 0.008*"training" + 0.006*"team" + 0.006*"build" + 0.006*"azure" + 0.006*"optimile" + 0.006*"applications" + 0.006*"flight"

Topic: 2 
Words: 0.015*"product" + 0.014*"warranty" + 0.014*"work" + 0.014*"team" + 0.009*"client" + 0.008*"marketing" + 0.008*"content" + 0.008*"company" + 0.007*"commission" + 0.007*"administrator"



**DATA SCIENTIST**

In [17]:
data_DS

,Category,Description
87,Data Scientist,Description\nData Science problems are everywh...
88,Data Scientist,Description\nAre you looking to make a measura...
89,Data Scientist,"Description\nAs a Data Scientist, you will wor..."
90,Data Scientist,Description\nAs a member of the Data team at T...
91,Data Scientist,Description\nThis is a data science position o...
...,...,...
185,Data Scientist,"Description\nCollect, prepare, and load files ..."
186,Data Scientist,NaN
187,Data Scientist,Description\nMedalogix is searching for a Prod...
188,Data Scientist,Description\nWho We Are:\nTickPick is a fast g...


In [18]:
# Drop rows with NaN values in the 'Description' column
data_DS = data_DS.dropna(subset=['Description'])

In [19]:
data_DS['Description']

87     Description\nData Science problems are everywh...
88     Description\nAre you looking to make a measura...
89     Description\nAs a Data Scientist, you will wor...
90     Description\nAs a member of the Data team at T...
91     Description\nThis is a data science position o...
                             ...                        
184    Description\nData Analysis: Perform advanced d...
185    Description\nCollect, prepare, and load files ...
187    Description\nMedalogix is searching for a Prod...
188    Description\nWho We Are:\nTickPick is a fast g...
189    Description\nData Analysis: Perform advanced d...
Name: Description, Length: 100, dtype: object

In [20]:
# Preprocess the text data
def preprocess_text(text):
     # Check if text is empty or contains only a single token
    if not text or len(text) == 1:
        return []
    
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens


# Apply preprocessing to the Description column
data_DS['Description'] = data_DS['Description'].apply(preprocess_text)


/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/326113863.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_DS['Description'] = data_DS['Description'].apply(preprocess_text)


In [21]:
data_DS['Description']

87     [description, data, science, problems, everywh...
88     [description, looking, make, measurable, impac...
89     [description, data, scientist, work, closely, ...
90     [description, member, data, team, trexquant, i...
91     [description, data, science, position, interna...
                             ...                        
184    [description, data, analysis, perform, advance...
185    [description, collect, prepare, load, files, d...
187    [description, medalogix, searching, product, o...
188    [description, tickpick, fast, growing, technol...
189    [description, data, analysis, perform, advance...
Name: Description, Length: 100, dtype: object

In [22]:
# Create bigrams and trigrams
bigram = Phrases(data_DS['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_DS['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_DS['Description'] = [bigram_mod[doc] for doc in data_DS['Description']]
data_DS['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_DS['Description']]

# Create a dictionary representation of the documents
dictionary_DS= corpora.Dictionary(data_DS['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_DS= [dictionary_DS.doc2bow(doc) for doc in data_DS['Description']]

# Train the LDA model
lda_model_DS = LdaModel(bow_corpus_DS, num_topics=3, id2word=dictionary_DS, passes=10)

# Print the topics
for idx, topic in lda_model_DS.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')
    
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_DS = gensimvis.prepare(lda_model_DS, bow_corpus_DS, dictionary_DS)

# Display the visualization
pyLDAvis.display(lda_vis_data_DS)


/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/1589208032.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_DS['Description'] = [bigram_mod[doc] for doc in data_DS['Description']]
/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/1589208032.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_DS['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_DS['Description']]


Topic: 0 
Words: 0.044*"data" + 0.014*"team" + 0.013*"business" + 0.008*"work" + 0.008*"models" + 0.007*"learning" + 0.006*"analysis" + 0.006*"develop" + 0.006*"new" + 0.006*"machine"

Topic: 1 
Words: 0.030*"data" + 0.011*"team" + 0.011*"analytics" + 0.011*"business" + 0.008*"science" + 0.006*"projects" + 0.005*"role" + 0.005*"key" + 0.005*"tools" + 0.005*"ensure"

Topic: 2 
Words: 0.053*"data" + 0.011*"business" + 0.010*"product" + 0.008*"team" + 0.008*"work" + 0.007*"teams" + 0.006*"science" + 0.005*"development" + 0.005*"description" + 0.005*"engineering"



**HR**

In [23]:
data_hr

,Category,Description
190,HR,Description\nNew Employee Onboarding:\n· ...
191,HR,Description\nLibrary Systems & Services (LS&S)...
192,HR,Description\nNew Employee Onboarding:\n· ...
193,HR,Description\nLibrary Systems & Services (LS&S)...
194,HR,Description\nWe are a US-based company seeking...
195,HR,"Description\nDangote Group, one of Nigeria’s m..."
196,HR,"Description\nWe're Plentific, the world’s lead..."
197,HR,"Description\nAbout us\nLove, Bonito is a digit..."
198,HR,Description\nA HR Specialist shall be responsi...
199,HR,Description\nAbout us\nWith a long history of ...


In [24]:
# Drop rows with NaN values in the 'Description' column
data_hr = data_hr.dropna(subset=['Description'])

In [25]:
# Preprocess the text data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Apply preprocessing to the Description column
data_hr['Description'] = data_hr['Description'].apply(preprocess_text)

/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/959317425.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hr['Description'] = data_hr['Description'].apply(preprocess_text)


In [26]:
# Create bigrams and trigrams
bigram = Phrases(data_hr['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_hr['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_hr['Description'] = [bigram_mod[doc] for doc in data_hr['Description']]
data_hr['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_hr['Description']]

# Create a dictionary representation of the documents
dictionary_hr = corpora.Dictionary(data_hr['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_hr = [dictionary_hr.doc2bow(doc) for doc in data_hr['Description']]

# Train the LDA model
lda_model_hr= LdaModel(bow_corpus_hr, num_topics=3, id2word=dictionary_hr, passes=10)

# Print the topics
for idx, topic in lda_model_hr.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')
    
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_hr = gensimvis.prepare(lda_model_hr, bow_corpus_hr, dictionary_hr)

# Display the visualization
pyLDAvis.display(lda_vis_data_hr)


Topic: 0 
Words: 0.014*"team" + 0.014*"legal" + 0.012*"work" + 0.008*"join" + 0.007*"corporate" + 0.006*"ensuring" + 0.006*"business" + 0.006*"user" + 0.006*"across" + 0.006*"design"

Topic: 1 
Words: 0.013*"team" + 0.012*"support" + 0.010*"new" + 0.008*"hr" + 0.006*"development" + 0.006*"management" + 0.006*"work" + 0.006*"scrum" + 0.005*"issues" + 0.005*"employee"

Topic: 2 
Words: 0.021*"hr" + 0.012*"management" + 0.009*"employee" + 0.008*"business" + 0.008*"develop" + 0.007*"strategic" + 0.007*"people" + 0.007*"leadership" + 0.007*"development" + 0.007*"compensation"



/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/895795411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hr['Description'] = [bigram_mod[doc] for doc in data_hr['Description']]
/var/folders/wz/2r9dmd592pg7wwm18rccbml80000gn/T/ipykernel_6524/895795411.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hr['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_hr['Description']]


**SOFTWARE DEVELOPER**

In [27]:
data_SD

,Category,Description
222,Software Developer,Description\nWe are seeking a Software Develop...
223,Software Developer,Description\nFuturex is seeking talented indiv...
224,Software Developer,Description\nDesign (or redesigns) of our inte...
225,Software Developer,Description\nWe know that creating an inclusiv...
226,Software Developer,Description\nWe need a developer to help us wi...
...,...,...
310,Software Developer,"Description\nWe create powerful, beautiful and..."
311,Software Developer,Description\nRecognized as a leader in develop...
312,Software Developer,Description\nDo you want to be part of an amaz...
313,Software Developer,Description\nFounded in 2016 in Silicon Valley...


In [28]:
# Preprocess the text data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Apply preprocessing to the Description column
data_SD['Description'] = data_SD['Description'].apply(preprocess_text)

In [29]:
# Create bigrams and trigrams
bigram = Phrases(data_SD['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_SD['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_SD['Description'] = [bigram_mod[doc] for doc in data_SD['Description']]
data_SD['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_SD['Description']]

# Create a dictionary representation of the documents
dictionary_SD = corpora.Dictionary(data_SD['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_SD = [dictionary_SD.doc2bow(doc) for doc in data_SD['Description']]

# Train the LDA model
lda_model_SD = LdaModel(bow_corpus_SD, num_topics=3, id2word=dictionary_SD, passes=10)

# Print the topics
for idx, topic in lda_model_SD.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')
    
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_SD = gensimvis.prepare(lda_model_SD, bow_corpus_SD, dictionary_SD)

# Display the visualization
pyLDAvis.display(lda_vis_data_SD)


Topic: 0 
Words: 0.010*"software" + 0.007*"team" + 0.007*"design" + 0.007*"development" + 0.006*"support" + 0.006*"description" + 0.005*"applications" + 0.005*"ensure" + 0.005*"work" + 0.005*"product"

Topic: 1 
Words: 0.023*"software" + 0.010*"development" + 0.010*"team" + 0.007*"design" + 0.007*"code" + 0.006*"description" + 0.006*"technical" + 0.006*"solutions" + 0.005*"work" + 0.005*"role"

Topic: 2 
Words: 0.017*"software" + 0.012*"work" + 0.011*"team" + 0.009*"development" + 0.006*"description" + 0.006*"code" + 0.006*"clients" + 0.005*"project" + 0.005*"working" + 0.005*"ensure"



**UI/UX**

In [30]:
data_uiux

,Category,Description
315,UI/UX,Description\nJob Title: UI/UX Designer\nRequir...
316,UI/UX,Description\nThe UX Designer will be focused o...
317,UI/UX,Description\nWe are seeking a dynamic UI/UX de...
318,UI/UX,Description\nWe are looking for a UI/UX Design...
319,UI/UX,Description\nJob Title: UI/UX Designer\nRequir...
320,UI/UX,Description\nThe UX Designer will be focused o...
321,UI/UX,Description\nWe are seeking a dynamic UI/UX de...
322,UI/UX,Description\nWe are looking for a UI/UX Design...
323,UI/UX,Description\nRemote – Full-Time\nClay is a glo...
324,UI/UX,Description\nRemote – Full-Time\nClay is a glo...


In [31]:
# Preprocess the text data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Apply preprocessing to the Description column
data_uiux['Description'] = data_uiux['Description'].apply(preprocess_text)

In [32]:
# Create bigrams and trigrams
bigram = Phrases(data_uiux['Description'], min_count=5, threshold=100)
trigram = Phrases(bigram[data_uiux['Description']], min_count=5, threshold=100)
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
data_uiux['Description'] = [bigram_mod[doc] for doc in data_uiux['Description']]
data_uiux['Description'] = [trigram_mod[bigram_mod[doc]] for doc in data_uiux['Description']]

# Create a dictionary representation of the documents
dictionary_uiux = corpora.Dictionary(data_uiux['Description'])

# Convert the documents into a bag-of-words format
bow_corpus_uiux = [dictionary_uiux.doc2bow(doc) for doc in data_uiux['Description']]

# Train the LDA model
lda_model_uiux = LdaModel(bow_corpus_uiux, num_topics=3, id2word=dictionary_uiux, passes=10)

# Print the topics
for idx, topic in lda_model_uiux.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')
    
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis_data_uiux= gensimvis.prepare(lda_model_uiux, bow_corpus_uiux, dictionary_uiux)

# Display the visualization
pyLDAvis.display(lda_vis_data_uiux)


Topic: 0 
Words: 0.045*"user" + 0.030*"design" + 0.023*"web" + 0.016*"experience" + 0.012*"uiux" + 0.012*"collaborate" + 0.012*"understand" + 0.012*"marketing" + 0.008*"team" + 0.008*"work"

Topic: 1 
Words: 0.026*"design" + 0.018*"uiux" + 0.018*"digital" + 0.018*"clients" + 0.018*"team" + 0.018*"products" + 0.017*"senior" + 0.017*"worldclass" + 0.017*"clay" + 0.010*"user"

Topic: 2 
Words: 0.040*"design" + 0.030*"user" + 0.015*"experience" + 0.015*"ux" + 0.013*"designer" + 0.013*"uiux" + 0.013*"work" + 0.013*"designs" + 0.010*"team" + 0.010*"practices"

